### 사용자 입장에서 해야 할 step !
- G마켓 BEST상품의 1위부터 10위까지의 데이터를 수집
- 중요한 것 !
    - 첫 번째에 있는 상품을 클릭한 다음에 해당 상품의 이름과 가격 정보를 수집
    - 뒤로 간 다음 두 번째에 있는 상품을 클릭, 수집 똑같은 말을 반복 중
    - while ? for ? 어떤 것이 적합 ? -> for문 / 10번 정해져 있기 때문

In [1]:
# selenium 라이브러리 로딩

# 브라우저 역할 - 컴퓨터용 브라우저
from selenium import webdriver as wb

# 키보드 역할 (Keys K대문자 !!) - 컴퓨터용 키보드
from selenium.webdriver.common.keys import Keys

# 구분자 역할 (By B대문자 !!) - 선택자를 구분
from selenium.webdriver.common.by import By

# 수집한 이후 데이터프레임까지 만들어볼 예정
import pandas as pd

# 지연시간 설정
import time

In [2]:
driver = wb.Chrome()
url = 'https://corners.gmarket.co.kr/bestsellers'
driver.get(url)

#### 반복문을 시작하기 전 하나의 프로세스를 완벽하게 짜보기
#### 잘 완성되면 반복문 안에 프로세스를 통째로 넣어주기만 하면 됨!
1. 상품의 이미지를 클릭 (화면 변경)
> 사용자는 첫 번째 상품을 알 수 있지만 컴퓨터는 알 수 없다 !  
> 어떤 태그로 구성되어 있는지 검사해서 알려줘야 함
2. 상품의 이름과 가격을 수집
3. 페이지 뒤로가기 (화면 변경)

In [3]:
# 1. 상품의 이미지를 클릭

# 1-1. 이미지를 클릭하기 위해서 이미지 태그들을 알려줘야 함
imgs = driver.find_elements(By.CLASS_NAME, 'lazy')

# 1-2. 이미지 클릭
imgs[0].click()

# 요청을 복수개로 했으니 응답도 복수개이다 -> [리스트]

In [4]:
# 2-1. 상품명 수집

# 조회하고자 하는 요소가 class를 가지고 있을 경우에는
# 먼저 ! elements(복수)로 요청해본다.
# 그 다음에 데이터가 한 개만 넘어온다면, element(단일)로 수정해서 활용하면 편하다 !
title = driver.find_element(By.CLASS_NAME, 'itemtit')
title.text

'휴튼미플린하코트출판사가 만든 과학레벨드리더스 /Science and Engineering Leveled Readers'

In [5]:
# 2-2. 상품 가격 수집
# price 변수에 담아서 price.text 컨텐츠 추출

price = driver.find_element(By.CLASS_NAME, 'price_real')
price.text

'168,800원'

In [6]:
# 페이지 뒤로가기
driver.back()

In [9]:
# 1번부터 5번까지 데이터 수집

# 1. 비어있는 리스트에 수집한 텍스트 정보를 저장 (상품명, 가격)
# 2. 데이터프레임으로 생성
# 3. 참고 파일 -> 멜론차트, 영화리뷰 파일을 참고해서 완성해보자

title_list = []
price_list = []

for i in range(10) :
    
    imgs = driver.find_elements(By.CLASS_NAME, 'lazy')
    imgs[i].click()
    # 이미지를 클릭 했을 때 (화면 변화/깜빡) 1초 쉬어주기
    time.sleep(1)
    
    title = driver.find_element(By.CLASS_NAME, 'itemtit')
    title_list.append(title.text)
    
    price = driver.find_element(By.CLASS_NAME, 'price_real')
    price_list.append(price.text)
    
    driver.back()
    # 뒤로가기 실행했을 때(화면 변화/낌삑) 1초 쉬어주기
    time.sleep(1)
    
data = {'상품명':title_list,'가격':price_list}
pd.DataFrame(data)

,상품명,가격
0,휴튼미플린하코트출판사가 만든 과학레벨드리더스 /Science and Engineer...,"168,800원"
1,크로아상 크로플 생지 2250g (40~50개입) + 사과버터잼 100g 증정(쿠폰...,"21,100원"
2,더미식 백미밥 210g 24개+장인라면 컵라면 증정,"23,900원"
3,(깨비농원) 고당도 천혜향 4.5kg 소과(28-36과)/20%+5% 쿠폰,"30,900원"
4,태국 마하차녹 무지개 망고 2kg 9-10수 / 18680원 (25%쿠폰가) / 부...,SOLD OUT
5,에너자이저 알카라인 패밀리팩 건전지 AA24입+AAA24입,"27,000원"
6,고창 김민상님의 황토밭 햇 꿀밤고구마 5kg 특상 / 쿠폰가 12680 (3/29 하루),"16,900원"
7,훈제오리 슬라이스 800g + 800g,"23,600원"
8,(신세계센텀점)플랜팅 드로잉 스트라이프 티셔츠 OB 0016 카키,"30,870원"
9,탑텐 본사직영 쿨에어/티셔츠/폴로티/스웨터 외 특가,"9,290원"


### 크롤링의 순서 ! (프로세스)
- 1. 화면 구성 : 크롬창에 데이터를 수집 할 준비
- 2. 요소 수집 : 컴퓨터가 사용하는 HTML형태의 데이터 수집
- 3. 데이터 가공 : 수집된 요소에서 내가 필요한 정보만 추출
    - 3-1. 비어있는 리스트에 내가 필요한 정보를 저장
- 4. 데이터 활용 : 데이터프레임, 파일로 저장, DB로 저장